In [ ]:
import requests
import json
import pandas as pd

In [ ]:
df = pd.read_excel('.xlsx', sheet_name='Things')
df_csv = df.to_csv('.csv', index=False)

df_csv = pd.read_csv('.csv')
df_csv = df_csv.sort_values(by='unique Identifier\n(M)')
df_csv.head()

In [ ]:
df_col = pd.read_excel('.xlsx', sheet_name='Locations')
df_csv_col = df_col.to_csv('.csv', index=False)
df_csv_col = pd.read_csv('.csv')
df_csv_col = df_csv_col.sort_values(by='unique Identifier\n(M)')

df_csv_col.head()

In [ ]:
local_identifier = []

for index, row in df_csv_col.iterrows():
    unique_identifier = row["unique Identifier\n(M)"]
    room_ids = row["local identifier things\n(O)"].split(", ")

    if isinstance(unique_identifier, str):
        for room_id in room_ids:
            unique_identifier = unique_identifier.replace('OUT', '').replace('IND.', '')
            local_identifier.append((unique_identifier, room_id.strip()))



In [ ]:
local_identifier = []

for index, row in df_csv_col.iterrows():
    unique_identifier = row["unique Identifier\n(M)"]
    room_ids = row["local identifier things\n(O)"].split(", ")

    if isinstance(unique_identifier, str):
        for room_id in room_ids:
                unique_identifier = unique_identifier.replace('OUT', '').replace('IND.', '')
                local_identifier.append((unique_identifier, room_id.strip()))

iot_id_counter = 1

for index, row in df_csv.iterrows():
        
        json_data_thing = {
              "id": str(row["unique Identifier\n(M)"]),
              "method": "post",
              "url": "Things",
      "body":{
            "@iot.id": str(row["unique Identifier\n(M)"]),
            "name": str(row["name\n(M)"]),
            "description": str(row["description\n(M)"]),
            "properties": {
                "identifier": str(row["unique Identifier\n(M)"]),
                "owner": str(row["owner \n(O)"]),
                "beginDatetime": str(row["Begin datetime\n(M)"]),
                "endDatetime": str(row["End datetime\n(O)"]),
                "localIdentifier": str(row["local identifier\n(O)"])
            },
                "Locations": [] 
            }
        }
        

        # Scrivi il JSON in un file separato
        with open(f"json\\thing_{iot_id_counter}.json", "w", encoding='utf-8') as json_file:
            json.dump(json_data_thing, json_file, indent=4, ensure_ascii=False)

        iot_id_counter += 1


In [ ]:
for i in range(1, iot_id_counter): 
    with open(f"json\\thing_{i}.json", "r") as json_file:
        json_data_thing = json.load(json_file)

    location_to_add = None
    iot_id = f"THI..{iot_id_counter:03}" #sigla
    for item in local_identifier:
        if json_data_thing["body"]["properties"]["localIdentifier"] == item[1]:
            for idx, loc_row in df_csv_col.iterrows():
                if item[1] in loc_row["local identifier things\n(O)"]:  
                    iot_id = str(loc_row["unique Identifier\n(M)"])
                    
                    # Gestione dei valori 'nan'
                    def handle_nan(value):
                        if pd.isna(value):
                            if isinstance(value, str):
                                return "null"
                            else:
                                return 0
                        return value
                    
                    json_data_location = {
                        "@iot.id": str(iot_id),
                        "name": str(handle_nan(loc_row["name\n(M)"])),
                        "description": str(handle_nan(loc_row["description \n(M)"])),
                        "properties": {
                            "identifier": str(iot_id),
                            "isOutdoor" : "false", 
                            "owner": str(handle_nan(loc_row["owner \n(O)"])),
                            "idOfTheBuildingInOpenStreetMap": str(handle_nan(loc_row["ID of the building in OpenStreetMap\n(O)"])),
                            "address": str(handle_nan(loc_row["address\n(M)"])),
                            "country": str(handle_nan(loc_row["country \n(M)"])),
                            "useOfTheBuilding": str(handle_nan(loc_row["use of the building\n(M)"])),
                            "detailsAboutTheUse": str(handle_nan(loc_row["details about the use (O)"])),
                            "localIdTh": str(handle_nan(loc_row["local identifier things\n(O)"])),
                            "room": {
                                "idOfTheRoom": str(handle_nan(loc_row["ID of the room \n(O)"])),
                                "elevationOfTheThing": handle_nan(loc_row["Elevation of the thing (vertical distance from mean sea level) - (O)"]),
                                "levelFloor": handle_nan(loc_row["level / floor\n(O)"]),
                                "verticalDistanceFromPavementOrGround": handle_nan(loc_row["vertical distance  from pavement or ground (O)"]),
                                "verticalDistanceFromCeilingOfTheRoom": handle_nan(loc_row["vertical distance from ceiling of the room\n(O)"]),
                                "horizontalDistanceFromWalls": handle_nan(loc_row["horizontal distance from walls \n(O)"])
                            }
                        },
                        "encodingType": "application/geo+json",
                        "location": {
                            "type": "Point",
                            "coordinates": [handle_nan(loc_row["Longitude coordinate \n(O)"]), handle_nan(loc_row["Latitude coordinate \n(O)"])]
                        }
                    }

                    
                    location_to_add = json_data_location
                    break


    if location_to_add:
        json_data_thing["body"]["Locations"] = [location_to_add]


    with open(f"json\\thing_{i}.json", "w", encoding='utf-8') as json_file:
        json.dump(json_data_thing, json_file, indent=4, ensure_ascii=False)

In [ ]:
#BATCH

json_struct = []

fixed_part = {
    "POST to": "v1.1/$batch",
    "Headers": "Content-Type: application/json",
    "requests": []  
}
json_struct.append(fixed_part)

In [ ]:
for i in range(1, iot_id_counter):  # Loop su tutti i file JSON creati precedentemente
    with open(f"json\\thing_{i}.json", "r") as json_file:
        json_data_thing = json.load(json_file)
        json_struct[0]["requests"].append(json_data_thing)

In [ ]:
formatted_json = json.dumps(json_struct, indent=3, ensure_ascii=False)

try:
    json.loads(formatted_json)
    print("Il JSON è stato formattato correttamente.")
except json.JSONDecodeError as e:
    print(f"Errore nella formattazione: {e}")
    
with open('complete_th_loc_prova.json', 'w', encoding='utf-8') as json_file:
    json_file.write(formatted_json)
